In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [ ]:
def classify_examples(X, Y, k = 3):
    """
    Classify examples as noisy or safe based on their k nearest neigbors.

    Args:
        X (numpy.ndarray): The feature matrix of shape (n_samples, n_features)
        Y (numpy.ndarray): The target labels of shape(n_samples,)
        k (int, optional): The number of nearest neighbors to consider, defaults to 3
    
    Returns:
        numpy.ndarray: An array of flags indicating the type of each example (0 for safe, 1 for noisy)
    """
    n_samples = X.shape[0]
    flags = np.zeros(n_samples, dtype=int)  # Intialize flags for all examples as safe

    # Fit a k-nearest neighbors model
    nn = NearestNeighbors(n_neighbors=k)
    nn.fit(X)

    for i in range(n_samples):
        example = X[i]
        label = Y[i]

        # Find the indices of the k nearest neigbors
        indices = nn.kneighbors([example], return_distance=False)

        # Exclude the current example itself from the neigbors
        neighbors_indices = indices[0][1:]

        # Check if the majority of neighbors have the same label as the current example
        if np.sum(Y[neighbors_indices] == label) >= k // 2:
            flags[i] = 0    # Set flag as safe
        else:
            flags[i] = 1    # Set flag as noisy
    return flags

In [ ]:

X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([0, 1, 1, 1])
classify_examples(X, y, k=3)